In [ ]:
!pip install opencv-python
!pip install numpy
!pip install pyzbar

In [ ]:
'''
made by 안산드레이아스
https://ansan-survivor.tistory.com/

'''


In [1]:
import cv2
import numpy as np
import pyzbar.pyzbar as pyzbar
from tkinter import Tk, filedialog
import shutil
import os

# 바코드 탐지하는 엔진 (바코드 및 QR코드 탐지)
def decode(im):
    # Find barcodes and QR codes
    decodedObjects = pyzbar.decode(im)

    # Print results
    for obj in decodedObjects:
        print('Type: ', obj.type)
        print('Data: ', obj.data, '\n')

    return decodedObjects


# Display barcode and QR code location
def display(im, decodedObjects):
    # Loop over all decoded objects
    for decodedObject in decodedObjects:
        points = decodedObject.polygon

        # If the points do not form a quad, find convex hull
        if len(points) > 4:
            hull = cv2.convexHull(np.array([point for point in points], dtype=np.float32))
            hull = list(map(tuple, np.squeeze(hull)))
        else:
            hull = points

        # Number of points in the convex hull
        n = len(hull)

        # Draw the convex hull
        for j in range(0, n):
            cv2.line(im, tuple(map(int, hull[j])), tuple(map(int, hull[(j + 1) % n])), (255, 0, 0), 3)  # 코드 에러있어서 수정

    # Display results
    cv2.imshow("Results", im)
    cv2.waitKey(0)


def select_file():
    Tk().withdraw()  # Hide the Tkinter main window
    file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg;*.jpeg;*.png")])
    return file_path


if __name__ == '__main__':
    # Select file
    file_path = select_file()

    if file_path:
        # Copy image file to 'dump' folder with the same extension
        folder_path = 'dump'
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)

        # Get the filename and extension
        file_name = os.path.basename(file_path)
        file_extension = os.path.splitext(file_name)[1]

        # Copy the file to 'dump' folder
        new_file_path = os.path.join(folder_path, file_name)
        shutil.copyfile(file_path, new_file_path)

        # Read the copied image file
        try:
            im = cv2.imread(new_file_path)

            if im is not None:
                decodedObjects = decode(im)
                display(im, decodedObjects)
            else:
                print("Failed to read the image.")
        except Exception as e:
            print("Error occurred while reading the image:", str(e))

        # Remove the copied image file
        os.remove(new_file_path)
        os.rmdir(folder_path)
    else:
        print("No file selected.")


Type:  CODE128
Data:  b'6100016300632' 

